In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read the data
INCIDENTS = '../dataset/data-raw/incidents.csv'
POVERTYYEAY = '../dataset/data-raw/povertyByStateYear.csv'
STATEDISHOUSE = '../dataset/data-raw/year_state_district_house.csv'

incidents = pd.read_csv(INCIDENTS)
poverty = pd.read_csv(POVERTYYEAY)
state_district_house = pd.read_csv(STATEDISHOUSE)

incidents['state'] = incidents['state'].str.upper()
poverty['state'] = poverty['state'].str.upper()
state_district_house['state'] = state_district_house['state'].str.upper()

/tmp/ipykernel_10094/2836909318.py:10: DtypeWarning: Columns (15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  incidents = pd.read_csv(INCIDENTS)


In [105]:
valid_districts = {}
for i in range(len(state_district_house)):
    state = state_district_house.loc[i, 'state']
    year = state_district_house.loc[i, 'year']
    district = state_district_house.loc[i, 'congressional_district']
    district = 1 if district == 0 else district
    if (state, year) not in valid_districts:
        valid_districts[(state, year)] = []
        valid_districts[(state, year+1)] = []
    valid_districts[(state, year)].append(district)
    valid_districts[(state, year+1)].append(district)

import math
def is_valid_district(state, year, district):
    if math.isnan(year):
        print('year is nan')
        return True
    if (state, year) in valid_districts:
        return district in valid_districts[(state, year)]
    return False

In [41]:
incidents.columns

Index(['date', 'state', 'city_or_county', 'address', 'latitude', 'longitude',
       'congressional_district', 'state_house_district',
       'state_senate_district', 'participant_age1', 'participant_age_group1',
       'participant_gender1', 'min_age_participants', 'avg_age_participants',
       'max_age_participants', 'n_participants_child', 'n_participants_teen',
       'n_participants_adult', 'n_males', 'n_females', 'n_killed', 'n_injured',
       'n_arrested', 'n_unharmed', 'n_participants', 'notes',
       'incident_characteristics1', 'incident_characteristics2'],
      dtype='object')

In [31]:
# Date cleaning
noerr_incidents = pd.DataFrame()
noerr_incidents['date'] = pd.to_datetime(incidents['date'])
noerr_incidents['date_year'] = noerr_incidents['date'].dt.year
noerr_incidents['date_month'] = noerr_incidents['date'].dt.month
noerr_incidents['date_day'] = noerr_incidents['date'].dt.day
noerr_incidents.drop(columns=['date'], inplace=True)

noerr_incidents['date_year'] = noerr_incidents['date_year'].map(lambda x: np.nan if x > 2018 else x)


In [106]:
noerr_incidents['state'] = incidents['state']
noerr_incidents['latitude'] = incidents['latitude']
noerr_incidents['longitude'] = incidents['longitude']
noerr_incidents['congressional_district'] = incidents['congressional_district']
noerr_incidents[noerr_incidents['congressional_district'] == 0] = 1
invalid_districts = ~noerr_incidents.apply(lambda row: is_valid_district(row['state'], row['date_year'], row['congressional_district']), axis=1)
# noerr_incidents.apply(lambda row: print(row), axis=1)

year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year is nan
year

In [118]:
a = noerr_incidents[invalid_districts][(~noerr_incidents[invalid_districts]['congressional_district'].isna()) * (noerr_incidents[invalid_districts]['state'] != 'DISTRICT OF COLUMBIA')]
b = noerr_incidents.iloc[1007]

In [60]:
# view columns <state, congressional_district> 
scd = noerr_incidents[['state', 'congressional_district']]
zeroCD = scd[scd['congressional_district'] == 0]['state'].unique()
scd[scd['state'].isin(zeroCD)].groupby('state').describe()

congressional_district                                          \
                              count      mean       std  min  25%  50%  75%   
state                                                                         
DELAWARE                     1629.0  0.882136  0.333776  0.0  1.0  1.0  1.0   
MONTANA                       617.0  0.857374  0.349974  0.0  1.0  1.0  1.0   
NORTH DAKOTA                  546.0  0.904762  0.293813  0.0  1.0  1.0  1.0   
SOUTH DAKOTA                  516.0  0.881783  0.323178  0.0  1.0  1.0  1.0   
WYOMING                       487.0  0.948665  0.220906  0.0  1.0  1.0  1.0   

                   
              max  
state              
DELAWARE      4.0  
MONTANA       1.0  
NORTH DAKOTA  1.0  
SOUTH DAKOTA  1.0  
WYOMING       1.0

,state,congressional_district
533,DELAWARE,1.0
593,DELAWARE,1.0
822,DELAWARE,1.0
953,DELAWARE,1.0
1170,DELAWARE,1.0
...,...,...
237902,DELAWARE,1.0
238001,DELAWARE,1.0
238390,DELAWARE,1.0
239259,DELAWARE,1.0
